In [15]:
import pandas as pd
import re

In [231]:
df = pd.read_csv('../../data/dataset_from_json.csv')
df.head()

,words,sentence #,tag
0,CPT,0,Incoterms
1,LAHORE,0,Incoterms
2,AIRPORT,0,Incoterms
3,PAKISTAN,0,Incoterms
4,QTY,0,O


In [232]:
df.iloc[115,:]

words         31.10.2019
sentence #             5
tag                    O
Name: 115, dtype: object

In [233]:
combine = lambda x: ' '.join(x['words'])

In [234]:
words = df.groupby(df['sentence #']).apply(combine)
words

sentence #
0       CPT LAHORE AIRPORT PAKISTAN QTY 1 PC OF MULTI ...
1       SERVICES AND MAINTENANCE AS PER BENEFICIARY'S ...
2       SERVICES MAINTENANCE AS PER BENEFICIARY'S PROF...
3       CPT LAHORE AIRPORT PAKISTAN PHARMACEUTICAL ALL...
4       CFR KEAMARI KARACHI SEAPORT/PORT QASIM PAKISTA...
                              ...                        
1615    CFR KARACHI PORT QTY 1000 KGS MUSK AMBRETTE (C...
1616    BY VESSEL (1) BENEFICIARY'S MANUALLY SIGNED OR...
1617    CFR KARACHI PORT / PORT QASIM PAKISTAN (1) QTY...
1618    CFR KARACHI SEAPORT.....TEST GOODS ARE IMPORTE...
1619    FCA HONG KONG QTY 03 PCS USG6630E-ACFIREWALL A...
Length: 1620, dtype: object

In [235]:
# example
words[5]

"CFR KARACHI SEAPORT PAKISTAN DISCOFIX / DOSIFIX AS PER BENEFICIARY'S PROFORMA INVOICE NO. 1807391726 DATED 31.10.2019"

In [236]:
find_special = lambda x: '58/60' in x #'CFR KARACHI PORT/ PORT QASIM TOTAL QUANTITY 21 MT PARAFFIN WAX 58/60 FULLY REFINED AT USD 950.00 PER M/TON AS PER INDENT NO. SBR/DEPT.C/581/2019 DATED 19.11.19 OD M/S S.B.R. AND CO. KARACHI PAKISTAN'
find_special = lambda x: '950.00' in x 

specials = words[words.apply(find_special)]
specials

sentence #
177     CFR KARACHI PORT 1) QTY 49.50 M/TONS POLYPROPY...
201     CFR PORT QASIM PAKISTAN 1- QTY 1800.00 KGS EXO...
226     CFR ANY PORT IN PAKISTAN QTY 247.50 M/TON PP 5...
228     CFR ANY PORT IN PAKISTAN QTY 247.50 M/TON PP 5...
287     CFR KARACHI PORT/ PORT QASIM TOTAL QUANTITY 21...
608     CFR KEAMARI KARACHI PORT PAKISTAN QTY 25.50 M....
615     CFR KARACHI SEAPORT TOTAL QUANTITY 21 MT PARAF...
763     CFR KEAMARI KARACHI PORT PAKISTAN QTY 76.50 M....
843     CFR KEAMARI KARACHI SEAPORT PAKISTAN 1-QTY 153...
849     CFR KEAMARI KARACHI PORT PAKISTAN 1- QTY 76.50...
947     CFR KARACHI SEAPORT PAKISTAN MACHINERY FOR DAI...
1360    CFR KARACHI PORT PAKISTAN 1-QTY 4 PCS OF RICE ...
1469    CFR ANY PORT IN PAKISTAN (1) QTY 792.00 M/TONS...
1531    CFR KEAMARI KARACHI PORT PAKISTAN QTY 76.50 M....
dtype: object

In [237]:
# we need \S instead of \w because we have dates: 31.10.2019 which need to be extracted as a single entity

# EXAMPLE
# extract_containing_numbers = lambda x: re.findall(r'\d+\S+', x)
# extract_containing_numbers
# words.apply(extract_containing_numbers).tolist()

In [238]:
#extract the NUMBERS, INCOTERMS and DATES for better tokenisation
extract_numbers = lambda x: re.findall(r'\d+\S+', x)
list_words = words.apply(extract_numbers).tolist()
flat_list = [el for l in list_words for el in l]

In [246]:
words
words.apply(extract_numbers).tolist()
len(flat_list)

11140

In [66]:
numbers = set(flat_list)
numbers

# we now need to remove ONLY the . at the end
numbers = [n[:-1] if n[-1] == '.' else n for n in list(numbers)]

Now we need to classify them in 3 categories: 
1. \<number\> for Quantity and UnitPriceAmount example 2000
2. \<ID\> for INCOTERMS example 2019H29Q2
3. \<DATE\> for improved tokenisation

Why? We have a new date everyday, and if we include every day as another date in the vocabulary we might think eroneously that we have a large vocabulary, when in fact all we have is a lot of numbers, ids and dates.

In [117]:
nrs = pd.Series(numbers) #convert to Series for ease of use
nrs.head(), nrs.shape

(0       58/60
 1       84.65
 2     22.4785
 3        9299
 4    29.11.19
 dtype: object,
 (5310,))

I am speculating that

1. Quantity contains only Integer values: 1000, 200, 7000
2. UnitPriceAmount contains Float values: 250.00, 293.11
3. Dates have a specific type: xx/xx/xxxx or xx-xx-xxxx
4. Everything else is Incoterms

In [156]:
# Quantity
def convert_to_int(x):    
    try:
        int(x)
        return True
    except ValueError:
        return False

quantities = nrs.apply(convert_to_int)

In [154]:
# UnitPriceAmount
def convert_to_float(x):    
    try:
        if not convert_to_int(x):
            float(x)
        else:
            return False

        return True
    except ValueError:
        return False

units = nrs.apply(convert_to_float)

In [209]:
# Dates
def convert_to_ids(x):    
    try:
        is_date = re.findall('^\d{2}(\.|\/)\d{2}(\.|\/)\d{4}$', x)
        if len(is_date) == 0:
            return False
        else:
            return True
    except ValueError:
        return False

dates = nrs.apply(convert_to_ids)

In [210]:
nrs[dates]

8       19.12.2019
26      01.11.2019
57      08.01.2019
116     12.09.2019
218     09/11/2019
           ...    
5085    23.09.2019
5142    03.03.2020
5241    26.12.2019
5248    05.08.2019
5275    17/12/2019
Length: 143, dtype: object

In [228]:
# the rest are incoterms
incoterms = ~quantities & ~units & ~dates
incoterms

0        True
1       False
2       False
3       False
4        True
        ...  
5305     True
5306    False
5307     True
5308    False
5309     True
Length: 5310, dtype: bool

In [229]:
quantities.sum() + units.sum() + dates.sum() + incoterms.sum(), nrs.shape[0]

(5310, 5310)

In [249]:
nrs[quantities].tolist()

['9299',
 '8600',
 '1030',
 '20016886',
 '191220',
 '19112601',
 '20200239',
 '300060',
 '307',
 '72',
 '3406',
 '1150',
 '58800',
 '701',
 '20191230',
 '636191013',
 '8158052384',
 '1040',
 '204',
 '4246',
 '23478',
 '46356242',
 '3100049685',
 '990',
 '191219',
 '1523121',
 '19',
 '001',
 '05870',
 '2013',
 '19001340',
 '004052',
 '13000',
 '191124',
 '1840',
 '9003',
 '200117',
 '19000',
 '2644',
 '11250',
 '240',
 '19001500',
 '15000',
 '5040',
 '2018',
 '802026895',
 '6400',
 '8102856808',
 '9400',
 '9228',
 '28517',
 '72000',
 '17860',
 '5100531666',
 '525610',
 '8020',
 '190121401',
 '12089',
 '20030000001353',
 '10200',
 '41362',
 '17',
 '4400005086',
 '78991200',
 '725000',
 '993143007',
 '1110000',
 '132000',
 '315',
 '12000',
 '6174071779',
 '1016',
 '7280044634',
 '11',
 '20001',
 '3131',
 '003',
 '20011301',
 '23171',
 '1108800',
 '320',
 '294',
 '6260',
 '2150',
 '4400025595',
 '003576',
 '19222037',
 '73',
 '2600',
 '1030049',
 '11171040',
 '025298',
 '9113001',
 '1000',

In [260]:
dictionary = {}

def append_to_dict(key, value):
    dictionary[key] = value


nrs[quantities].apply(append_to_dict, args=('<QUANTITY>',))
nrs[units].apply(append_to_dict, args=('<UNITPRICEAMOUNT>',))
nrs[dates].apply(append_to_dict, args=('<DATE>',))
nrs[incoterms].apply(append_to_dict, args=('<INCOTERMS>',))

0       None
4       None
5       None
10      None
11      None
        ... 
5296    None
5300    None
5305    None
5307    None
5309    None
Length: 2177, dtype: object

In [267]:
# now we need to map through the dataset and change the values.

def mapping(word):
    if word in dictionary.keys():
        return dictionary[word]
    return word

df['new_mappings'] = df['words'].apply(mapping)

In [273]:
df[(df['words'] != df['new_mappings'])]

,words,sentence #,tag,new_mappings
5,1,0,Quantity,<QUANTITY>
23,201912Q016H,1,O,<INCOTERMS>
25,05.12.2019,1,O,<DATE>
34,20193264,2,O,<QUANTITY>
36,03.12.2019,2,O,<DATE>
...,...,...,...,...
60300,01-04-2019,1617,O,<INCOTERMS>
60322,03,1619,Quantity,<QUANTITY>
60327,6790.56,1619,UnitPriceAmount,<UNITPRICEAMOUNT>
60337,900659,1619,O,<QUANTITY>
